In [1]:
import json
from pytube import YouTube
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [2]:
json_path = '09-11-2021-11_43_23.json'
data = json.load(open(json_path, 'r'))

In [12]:
# download video clips into videos

for k, annot in data.items():
    url = annot['url']
    segs = annot['annotations']['segment']
    start = int(segs[0])
    end = int(segs[1])


    try:
        video = YouTube(url)
        yt_video = video.streams.get_highest_resolution()
        yt_video.download()
        clips = ffmpeg_extract_subclip("RAW Police protesters clash in downtown Denver.mp4", start, end)
        
    except:
        continue

    

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
